# Descargando los resultados electorales del 2023 de manera automatizada
**Autor: Nicolás Cardona - @cardonanl**

Este código fue creado para que cualquier persona con intereses en los resultados electorales de Colombia del 2023 pueda acceder a ellos a nivel de puesto de votación de manera automática.

Es importante que el usuario NO modique nada que el código no sugiera modificar para evitar desde malos resultados hasta el colapso del servidor de la Registraduria.

Lo más importante para el usuario es elegir desde un inicio cuál nivel electoral va a minar y de qué departamento. Basado en un código único y un prefijo único, cuya obtención podrán ver en el GitHub, el usuario puede descargar toda la información de todos los candidatos y todos los puestos de votación.

# Puestos de votacion


*   Para extraer los votos se necesita tener primero los puestos de votación.
*   El usuario solo debe modificar un valor en todo el código para obtener los puestos de votación deseados.
*   Para encontrar el código, remitase el tutorial de GitHub.



In [ ]:
# Este codigo accede a un documento llamado nomenclator.json que hace parte del backend del preconteo de la Registraduria
# Este documento es algo complejo y aún no lo descifro bien porque su visualización es algo compleja debido a su extensión
# Adicionalmente, la estructura de datos en su interior tiene seis estructuras similares, es decir, al buscar mi puesto de votación de prueba -31001010107- este aparecía seis veces en una estructura similar.
# Parece ser que el documento contiene una estructura basada en el nivel de la votación.
# De cualquier manera; mi interés era encontrar todos los puestos de votación de un departamento o municipio por lo cual este codigo lo que hace es separar el primer matcha basado en un prefijo que son los tres primeros numeros del json que contiene los votos
# Ejemplo: en el puesto de Cali que use para testear (31001010107) el 310 al inicio indica Valle del Cauca. Así, la función va a devolver todos los puestos según ese valor.
# El usuario solo debe modificar un valor en el código

import requests

url = 'https://resultadosprec2023.registraduria.gov.co/json/nomenclator.json'

#Obtener el archivo
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

import json


#Función para extraer puestos de votación sin duplicados
def extraer_puestos_unicos(data, prefijo):
    puestos = {}
    for item in data['ambitos']['3']:
        # Acceder al primer grupo de datos relevante aquí
        codigo = item.get('c', '')
        if codigo.startswith(prefijo):
            # Usar el código como clave evita duplicados automáticamente
            puestos[codigo] = item

    return list(puestos.values())

#Ejecutar funcion
puestos_unicos = extraer_puestos_unicos(data, '400') # <--------- Este es el único valor que el usuario debe modificar

#Imp
for puesto in puestos_unicos:
    print(puesto)

In [ ]:
# El mismo código anterior pero usando nomenclator.json como archivo local. Esta version se comparte por si el link de la Registraduria deja de funcionar y el archivo se encontrará con los demás.

import json

#Cargar  JSON
with open('nomenclator.json', 'r') as archivo:
    data = json.load(archivo)

#Función para extraer puestos de votación sin duplicados
def extraer_puestos_unicos(data, prefijo):
    puestos = {}
    for item in data['ambitos']['3']:
        codigo = item.get('c', '')
        if codigo.startswith(prefijo):
            puestos[codigo] = item

    return list(puestos.values())

#Ejecutar funcion
puestos_unicos = extraer_puestos_unicos(data, '400')

#Imp
for puesto in puestos_unicos:
    print(puesto)


In [ ]:
#Pequeño código para convertir los resultados en un dataframe y así exportarlo más facilmanete si se precisa
import pandas as pd
df_puestos = pd.DataFrame(puestos_unicos)
print(df_puestos.head())
df_puestos

# Código para extraer votos de toda una serie de


*   Este código usa directamente la lista de puestos obtenidos para iterar sobre cada uno y extraer los votos por candidato y puesto de votación
*   Aca el usuario solo debe cambiar el link 'https://resultadosprec2023.registraduria.gov.co/json/ACT/AS/{codigo}.json' la parte del /ACT/AS así:
* Para Asamblea /ACT/AS - Para Concejos /ACT/CO - Para Alcaldía /ACT/AL - Para Gobernación /ACT/GO - Para JAL /ACT/JA
* **OJO:** Este proceso es intensivo en tiempo. Usando Google Colab, me tardó 1 hora descargando la info de la asamblea del Valle para todos los puestos de votación.



Votos - iteración final

In [ ]:
import requests
import pandas as pd
import time

# Lista de códigos únicos extraídos previamente
codigos_unicos = [puesto['c'] for puesto in puestos_unicos]

# Inicializar la lista para recolectar los datos
filas = []

# Iterar sobre cada código único
for codigo in codigos_unicos:
    url = f'https://resultadosprec2023.registraduria.gov.co/json/ACT/AS/{codigo}.json' # <- Este link puede ser modificado en su estructura para extraer los votos de las otras corporaciones /AS/ es asamblea por ejemplo.
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'camaras' in data:
            for camara in data['camaras']:
                if 'partotabla' in camara:
                    for partido in camara['partotabla']:
                        if 'cantotabla' in partido['act']:
                            for candidato in partido['act']['cantotabla']:
                                candidato['codpar'] = partido['act']['codpar']
                                candidato['codigo_unico'] = codigo
                                candidato['nombre_puesto'] = data.get('nombre_puesto', 'Desconocido')  # Ajustar clave si es necesario
                                filas.append(candidato)
        time.sleep(2)  # Esperar 2 segundos entre cada solicitud para no sobrecargar el servidor

# Crear DataFrame
df_final = pd.DataFrame(filas)
print(df_final.head(10))


In [ ]:
df_final.head(10)

In [ ]:
df_final.to_csv('votaciones_asamblea_arauca.csv')

Procesamiento adicional


*   Para mejorar la Base de Datos obtenida se pueden añadir el nombre de los partidos y el nombre de los puestos - municipios.
*   El código para los Partidos funciona para cualquier caso (tipo de elección o nivel electoral)
*   El código de los municipios es solo un ejemplo para el Valle del Cauca. Los nombres y códigos los pueden extraer directamente del mismo data frame obtenido en el código de los puestos de votación.



In [ ]:
################## Nombre del partido

mapeo_partidos = {
  '2':'Partido Liberal',
  '3':'Partido Conservador',
  '4':'Cambio Radical',
  '5':'Partido Verde',
  '9':'Partido de la U',
  '12':'Centro Democrático',
  '16':'Partido Colombia Renaciente',
  '24':'Partido Liga Gobernantes Anticorrupción',
  '26':'Partido Ecologista Colombiano',
  '36':'Movimiento Político Fuerza Ciudadana',
  '1773':'Fuerza de la Paz-Mais',
  '1862':'Valle Nos Une',
  '2803':'Pacto Histórico',
  '2997':'Acuerdo de Coalición',
}

df_final['Partido'] = df_final['codpar'].apply(lambda x: mapeo_partidos.get(x, 'Desconocido'))

In [ ]:
################## Municipios y nombre del puesto de votacion

# Eliminar la columna 'nombre_puesto' si existe
if 'nombre_puesto' in df_final.columns:
    df_final.drop('nombre_puesto', axis=1, inplace=True)

# Crear un diccionario para mapear 'c' a 's' de codigos_unicos
mapeo_s = {puesto['c']: puesto['s'] for puesto in puestos_unicos}

# Añadir la columna 'slug_puesto' basada en 'codigo_unico'
df_final['slug_puesto'] = df_final['codigo_unico'].apply(lambda x: mapeo_s.get(x, 'Desconocido'))

# Diccionario para mapear los códigos de municipio a sus nombres
mapeo_municipios = {
    "31004": "ALCALA",
    "31007": "ANDALUCIA",
    "31010": "ANSERMANUEVO",
    "31013": "ARGELIA",
    "31016": "BOLIVAR",
    "31019": "BUENAVENTURA",
    "31022": "BUGA",
    "31025": "BUGALAGRANDE",
    "31028": "CAICEDONIA",
    "31001": "CALI",
    "31040": "CALIMA (DARIEN)",
    "31031": "CANDELARIA",
    "31034": "CARTAGO",
    "31037": "DAGUA",
    "31043": "EL AGUILA",
    "31046": "EL CAIRO",
    "31049": "EL CERRITO",
    "31052": "EL DOVIO",
    "31055": "FLORIDA",
    "31058": "GINEBRA",
    "31061": "GUACARI",
    "31064": "JAMUNDI",
    "31067": "LA CUMBRE",
    "31070": "LA UNION",
    "31073": "LA VICTORIA",
    "31076": "OBANDO",
    "31079": "PALMIRA",
    "31082": "PRADERA",
    "31085": "RESTREPO",
    "31088": "RIOFRIO",
    "31091": "ROLDANILLO",
    "31094": "SAN PEDRO",
    "31097": "SEVILLA",
}


#Extraer los primeros 5 dígitos de 'codigo_unico' para obtener el código del municipio
df_final['municipio'] = df_final['codigo_unico'].apply(lambda x: mapeo_municipios.get(x[:5], 'Desconocido'))

print(df_final.head())


     amb codcan      cedula             nomcan             apecan nomcan2  \
0  31004     51    31540683             LUZDEY  MARTINEZ MARTINEZ           
1  31004     57    16452965     CARLOS ALBERTO  BEJARANO CASTILLO           
2  31004     71  1112957291         JORGE LUIS       REYES TASCON           
3  31004     55  1130670768           JONATHAN   VELASQUEZ ALZATE           
4  31004      0              SOLO POR LA LISTA                              

  apecan2  vot    pvot carg pref empate codpar codigo_unico slug_puesto  \
0          597  11,44%    0    1      0      2        31004      ALCALA   
1          505   9,68%    0    1      0      2        31004      ALCALA   
2           99   1,89%    0    1      0      2        31004      ALCALA   
3           95   1,82%    0    1      0      2        31004      ALCALA   
4           73   1,39%    0    1      0      2        31004      ALCALA   

  municipio  
0    ALCALA  
1    ALCALA  
2    ALCALA  
3    ALCALA  
4    ALCALA  


In [ ]:
df_final.to_csv('votacion_asamblea_valle_final.csv')

Este es un código adicional y simple para extraer directamente los votos de un solo puesto de votación que se desea. Solo hay que cambiar la url.

In [ ]:
import requests
import pandas as pd

#Testeo
url = 'https://resultadosprec2023.registraduria.gov.co/json/ACT/AS/31001010107.json'

#Request
response = requests.get(url)
data = response.json()

#Preparar la lista para recolectar datos
filas = []

#Navegar en el JSON para encontrar 'cantotabla' - acá se distinguen los partidos
if 'camaras' in data:
    for camara in data['camaras']:
        if 'partotabla' in camara:
            for partido in camara['partotabla']:
                if 'cantotabla' in partido['act']:
                    for candidato in partido['act']['cantotabla']:
                        #Añadir el código del partido para cada fila
                        candidato['codpar'] = partido['act']['codpar']
                        filas.append(candidato)

#DF
df = pd.DataFrame(filas)
print(df.head())

           amb codcan    cedula             nomcan           apecan nomcan2  \
0  31001010107     51  94553087      CARLOS FELIPE      LOPEZ LOPEZ           
1  31001010107     69  16771449     DIEGO FERNANDO  MOSQUERA MOLINA           
2  31001010107      0            SOLO POR LA LISTA                            
3  31001010107     71  66680731         LINA MARIA    SEGURA MOYANO           
4  31001010107     68  75080624     HERMAN ENRIQUE     LOPEZ ALZATE           

  apecan2 vot    pvot carg pref empate codpar  
0          76  11,25%    0    1      0      9  
1          31   4,59%    0    1      0      9  
2          11   1,62%    0    1      0      9  
3           7   1,03%    0    1      0      9  
4           6   0,88%    0    1      0      9  
